<a href="https://colab.research.google.com/github/u-adrian/PSDA/blob/main/01_Exercise/Aufgabe_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:


from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Aufgabe 4: Korrelation und Multikollinearität

a) Lesen Sie den Datensatz Credit.csv ein. Erkunden Sie die Daten und konvertieren Sie
gegebenenfalls kategorische Variablen in ein geeignetes Format (z. B. mit pandas.get_dummies()
oder sklearn.model_selection.OneHotEncoder()). Die Zielvariable des Datensatz ist Balance.

b) Separieren Sie den Datensatz in 80%-20% für Training und Testdatensatz. Was müssen Sie hier beachten?

c) Erstellen Sie aus der Variable Ethnicity drei binäre Variablen Ethnicity_Caucasian,
Ethnicity_Asian und Ethnicity_African. Führen Sie eine lineare Regression mit OLS als
Schätzer mit diesen Variablen und allen anderen Variablen (außer Limit) durch. Berechnen Sie
Root Mean Square Error (RMSE) zwischen der Vorhersage und den wahren Werten im Test-
Datensatz. Beschreiben Sie Ihre Ergebnisse.

d) Führen Sie die Analyse in 4c) erneut durch, ohne die Variable Ethnicity_Caucasian. Vergleichen
Sie diese Ergebnisse mit denen aus 4c).

e) Führen Sie eine lineare Regression (OLS) mit folgenden Variablen durch:

i. Income, Limit, Age

ii. Income, Rating, Age

iii. Income, Limit, Rating, Age

Beschreiben Sie die Koeffizienten, Konfidenzintervalle und Signifikanzniveau. Vergleichen Sie die
Ergebnisse.

f) Was ist Korrelation? Wann benutzt man Pearson Korrelation, Spearman Korrelation und Kendalls
Tau? Berechnen Sie Korrelationen zwischen allen Variablen im Trainings-Datensatz. Welche
Schlussfolgerung können Sie daraus ziehen?

g) Führen Sie die Analyse in 4e) mit Random Forest anstatt der linearen Regression durch. Variieren
Sie dabei den Parameter random_state mit den Werten {1, 33, 135, 123, 99, 22} und
einigen selbstgewählten Werten. Berechnen Sie MSE des Test-Datensatzes. Vergleichen Sie
Feature Importance. Welche Schlussfolgerung können Sie daraus ziehen?

h) Was ist Multikollinearität? Was sind ihre möglichen Ursachen und Auswirkungen? Wie kann man
sie erkennen? Welche Maßnahmen gibt es, wenn Variablen in den Daten kollinear sind?

a) Lesen Sie den Datensatz Credit.csv ein. Erkunden Sie die Daten und konvertieren Sie gegebenenfalls kategorische Variablen in ein geeignetes Format (z. B. mit pandas.get_dummies() oder sklearn.model_selection.OneHotEncoder()). Die Zielvariable des Datensatz ist Balance.

In [71]:
import pandas as pd

In [72]:
df = pd.read_csv("drive/My Drive/PSDA/01_Exercise/dataset/Credit.csv",index_col=0)
columns = ['Gender','Student','Married']
for column in columns:
  df[column] = pd.factorize(df[column])[0]
df

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
1,14.891,3606,283,2,34,11,0,0,0,Caucasian,333
2,106.025,6645,483,3,82,15,1,1,0,Asian,903
3,104.593,7075,514,4,71,11,0,0,1,Asian,580
4,148.924,9504,681,3,36,11,1,0,1,Asian,964
5,55.882,4897,357,2,68,16,0,0,0,Caucasian,331
...,...,...,...,...,...,...,...,...,...,...,...
396,12.096,4100,307,3,32,13,0,0,0,Caucasian,560
397,13.364,3838,296,5,65,17,0,0,1,African American,480
398,57.872,4171,321,5,67,12,1,0,0,Caucasian,138
399,37.728,2525,192,1,44,13,0,0,0,Caucasian,0


In [73]:
X=df.drop(labels='Balance',axis=1)
y=df['Balance']

b) Separieren Sie den Datensatz in 80%-20% für Training und Testdatensatz. Was müssen Sie hier beachten?

**Antwort:** Hier gilt zu beachten, dass der Train-Test-Split randomisiert ist.

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=21)
X_train

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity
192,124.290,9560,701,3,52,17,1,1,1,Asian
279,128.040,6982,518,2,78,11,1,0,0,Caucasian
273,30.413,3690,299,2,25,15,1,1,1,Asian
129,123.299,8376,610,2,89,17,0,1,1,African American
286,18.967,1626,156,2,41,11,1,0,0,Asian
...,...,...,...,...,...,...,...,...,...,...
369,89.000,5759,440,3,37,6,1,0,1,Caucasian
49,44.522,2252,205,6,72,15,0,0,0,Asian
261,67.937,5184,383,4,63,12,0,0,0,Asian
313,27.825,5227,386,6,63,11,0,0,0,Caucasian


c) Erstellen Sie aus der Variable Ethnicity drei binäre Variablen Ethnicity_Caucasian, Ethnicity_Asian und Ethnicity_African. Führen Sie eine lineare Regression mit OLS als Schätzer mit diesen Variablen und allen anderen Variablen (außer Limit) durch. Berechnen Sie Root Mean Square Error (RMSE) zwischen der Vorhersage und den wahren Werten im Test- Datensatz. Beschreiben Sie Ihre Ergebnisse.

In [75]:
X_train, X_test = pd.get_dummies(X_train), pd.get_dummies(X_test)
X_train.rename(columns = {'Ethnicity_African American':'Ethnicity_African'}, inplace = True)
X_test.rename(columns = {'Ethnicity_African American':'Ethnicity_African'}, inplace = True)


X_train

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity_African,Ethnicity_Asian,Ethnicity_Caucasian
192,124.290,9560,701,3,52,17,1,1,1,0,1,0
279,128.040,6982,518,2,78,11,1,0,0,0,0,1
273,30.413,3690,299,2,25,15,1,1,1,0,1,0
129,123.299,8376,610,2,89,17,0,1,1,1,0,0
286,18.967,1626,156,2,41,11,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
369,89.000,5759,440,3,37,6,1,0,1,0,0,1
49,44.522,2252,205,6,72,15,0,0,0,0,1,0
261,67.937,5184,383,4,63,12,0,0,0,0,1,0
313,27.825,5227,386,6,63,11,0,0,0,0,0,1


In [85]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X_train.drop('Limit',axis=1), y_train)
y_pred = reg.predict(X_test.drop('Limit',axis=1))

mean_squared_error(y_test, y_pred, squared=False)

103.45462752456523

**Antwort:**  TODO

d) Führen Sie die Analyse in 4c) erneut durch, ohne die Variable Ethnicity_Caucasian. Vergleichen Sie diese Ergebnisse mit denen aus 4c).

In [88]:
reg = LinearRegression().fit(X_train.drop(['Limit','Ethnicity_Caucasian'],axis=1), y_train)
y_pred = reg.predict(X_test.drop(['Limit','Ethnicity_Caucasian'],axis=1))

mean_squared_error(y_test, y_pred, squared=False)

103.4546275245652

**Antwort:**

Durch das löschen von "Ethnicity_Caucasian" gehen keine Informationen verloren, da diese Spalte durch die 2 Spalten "Ethnicity_African" und "Ethnicity_Asian" rekonstruiert werden kann. Somit bleibt der Error (RMSE) unverändert.

e) Führen Sie eine lineare Regression (OLS) mit folgenden Variablen durch:

i. Income, Limit, Age

ii. Income, Rating, Age

iii. Income, Limit, Rating, Age

Beschreiben Sie die Koeffizienten, Konfidenzintervalle und Signifikanzniveau. Vergleichen Sie die Ergebnisse.

In [101]:
reg = LinearRegression().fit(X_train[['Income','Limit','Age']], y_train)
y_pred = reg.predict(X_test[['Income','Limit','Age']])

print('RMSE:',mean_squared_error(y_test, y_pred, squared=False))
print('coef:',reg.coef_)

RMSE: 157.9721897968653
coef: [-7.5088365   0.26438444 -0.69515214]


In [102]:
reg = LinearRegression().fit(X_train[['Income','Rating','Age']], y_train)
y_pred = reg.predict(X_test[['Income','Rating','Age']])

print('RMSE:',mean_squared_error(y_test, y_pred, squared=False))
print('coef:',reg.coef_)

RMSE: 163.6387061539073
coef: [-7.51127999  3.95374421 -0.73761642]


In [103]:
reg = LinearRegression().fit(X_train[['Income','Limit','Rating','Age']], y_train)
y_pred = reg.predict(X_test[['Income','Limit', 'Rating', 'Age']])

print('RMSE:',mean_squared_error(y_test, y_pred, squared=False))
print('coef:',reg.coef_)

RMSE: 161.77586735561243
coef: [-7.54326778  0.04555321  3.28201178 -0.7268333 ]


**TODO**

f) Was ist Korrelation? Wann benutzt man Pearson Korrelation, Spearman Korrelation und Kendalls Tau? Berechnen Sie Korrelationen zwischen allen Variablen im Trainings-Datensatz. Welche Schlussfolgerung können Sie daraus ziehen?


1.   Pearson Korrelation: Wird verwendet um den Lineare Zusammenhang zwischen zwei Variablen zu untersuchen.

2.   Spearman Korrelation: 
3.   Kendalls Tau





g) Führen Sie die Analyse in 4e) mit Random Forest anstatt der linearen Regression durch. Variieren Sie dabei den Parameter random_state mit den Werten {1, 33, 135, 123, 99, 22} und einigen selbstgewählten Werten. Berechnen Sie MSE des Test-Datensatzes. Vergleichen Sie Feature Importance. Welche Schlussfolgerung können Sie daraus ziehen?